In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
noteevents = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/NOTEEVENTS.csv")
diagnoses = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/DIAGNOSES_ICD.csv")
diagnoses_dict = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/D_ICD_DIAGNOSES.csv")
admissions = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/ADMISSIONS.csv")
patients = pd.read_csv("/project/projectdirs/m1532/Projects_MVP/_datasets/mimiciii/PATIENTS.csv")


/usr/common/software/python/3.7-anaconda-2019.10/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#DOD filtering
def getIndex(startStr,iterations, addE = True):
    start = int(startStr)
    filtered_index = []
    index = []
    for x in range(iterations):
        if addE:
            filtered = diagnoses_dict["ICD9_CODE"].str.startswith("E"+startStr,na=False)
            start = start +1
            startStr = str(start)
        else:
            filtered = diagnoses_dict["ICD9_CODE"].str.startswith(startStr,na=False)
            start = start +1
            startStr = str(start)
        filtered_index = filtered[filtered].index
        index.append(filtered_index.tolist())
    return [inde for sublist in index for inde in sublist]
        
flat_filtered_list = []

In [22]:
#1.1
flat_filtered_list.append(getIndex("961",6, addE=True))
flat_filtered_list.append(getIndex("9680",10, addE=True))
flat_filtered_list.append(getIndex("967",10, addE=True))
flat_filtered_list.append(getIndex("990",100, addE=True))
flat_filtered_list.append(diagnoses_dict[diagnoses_dict["ICD9_CODE"].isin(["E9601","E969"])].index.tolist())

#1.2
flat_filtered_list.append(getIndex("630",50, addE=False))
len(flat_filtered_list)

#1.3
flat_filtered_list.append(getIndex("950",10, addE=True))
flat_filtered_list.append(getIndex("960",20, addE=False))
flat_filtered_list.append(getIndex("980",10, addE=True))
flat_filtered_list[-1]

#1.4
flat_filtered_list.append(getIndex("291",2, addE=False))
flat_filtered_list.append(getIndex("303",3, addE=False))
flat_filtered_list.append(getIndex("312",1, addE=False))
flat_filtered_list.append(getIndex("571",1, addE=False))
flat_filtered_list[-1]

#1.5
flat_filtered_list.append(getIndex("327",1, addE=False))
flat_filtered_list.append(getIndex("338",2, addE=False))
flat_filtered_list.append(getIndex("346",1, addE=False))
flat_filtered_list.append(getIndex("295",4, addE=False))
flat_filtered_list.append(getIndex("300",3, addE=False))
flat_filtered_list.append(getIndex("307",1, addE=False))
flat_filtered_list.append(getIndex("309",3, addE=False))
flat_filtered_list.append(getIndex("313",1, addE=False))

flat_filtered_list[-1]

In [ ]:
#getting specific ICD9_CODEs
icd9_code = []
x=0
for sublist in flat_filtered_list:
    for val in sublist:
        x+=1
        icd9_code.append(str(diagnoses_dict.iloc[val]["ICD9_CODE"]))

print(icd9_code)

In [24]:
diagnoses_df = diagnoses[diagnoses["ICD9_CODE"].isin(icd9_code)]
diagnoses_df

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
22,1319,109,173633,9.0,32723
24,1321,109,173633,11.0,33829
41,1501,114,178393,6.0,3051
63,1523,117,140784,1.0,5715
74,1534,117,164853,3.0,5712
...,...,...,...,...,...
651009,639765,97488,161999,10.0,29181
651017,639773,97488,161999,18.0,3051
651020,639776,97488,161999,21.0,30391
651024,639780,97492,189314,1.0,34680


In [25]:
#create 1 with addition of DOB, admittime, dischtime
addDOB = diagnoses_df.merge(patients[["SUBJECT_ID","DOB"]], on = "SUBJECT_ID")
final_df = addDOB.merge(admissions[["SUBJECT_ID","HADM_ID","ADMITTIME","DISCHTIME"]], on = ["SUBJECT_ID","HADM_ID"], how = "left")

final_df = final_df.drop_duplicates(subset = ["SUBJECT_ID","HADM_ID"]).drop("SEQ_NUM", axis=1)
final_df.to_pickle("final_df.pickle")

In [4]:
#while loading = 1
final_df = pd.read_pickle("pickles/final_df.pickle")

In [26]:
#while loading = 2
notes_df = noteevents[["SUBJECT_ID","HADM_ID","CHARTDATE","CHARTTIME","CATEGORY","TEXT"]]
notes_df = final_df[["HADM_ID","SUBJECT_ID","DOB"]].drop_duplicates().merge(notes_df, on=["SUBJECT_ID","HADM_ID"], how="right")

notes = noteevents[["SUBJECT_ID","HADM_ID","CHARTDATE","CHARTTIME","CATEGORY","TEXT"]]
notes_df = pd.merge(final_df[["SUBJECT_ID","HADM_ID","DOB"]],notes.drop_duplicates(subset = ["SUBJECT_ID","HADM_ID","CHARTDATE","CHARTTIME"]), on=["SUBJECT_ID","HADM_ID"], how = "left")
notes_df = notes_df.drop_duplicates(subset='TEXT')
notes_df


,SUBJECT_ID,HADM_ID,DOB,CHARTDATE,CHARTTIME,CATEGORY,TEXT
0,109,173633,2117-08-07 00:00:00,2141-12-14,NaN,Discharge summary,Admission Date: [**2141-12-8**] ...
1,109,173633,2117-08-07 00:00:00,2141-12-07,NaN,ECG,Sinus tachycardia. No significant change comp...
2,109,173633,2117-08-07 00:00:00,2141-12-11,NaN,ECG,Sinus rhythm with first degree A-V block. Q-T...
3,109,173633,2117-08-07 00:00:00,2141-12-08,NaN,ECG,Sinus rhythm. No significant change compared ...
4,109,173633,2117-08-07 00:00:00,2141-12-09,2141-12-09 03:12:00,Nursing,Ms [**Known lastname 406**] is a 24 year old w...
...,...,...,...,...,...,...,...
370137,97492,189314,2126-09-04 00:00:00,2158-08-01,2158-08-01 10:17:00,Rehab Services,TITLE:\nBEDSIDE SWALLOWING EVALUATION:\nHISTOR...
370138,97492,189314,2126-09-04 00:00:00,2158-08-01,2158-08-01 00:34:00,Physician,SICU\n HPI:\n 31 yo RH woman with a PMH de...
370139,97492,189314,2126-09-04 00:00:00,2158-08-01,2158-08-01 10:43:00,Radiology,[**2158-8-1**] 10:43 AM\n MR HEAD W/O CONTRAST...
370140,97492,189314,2126-09-04 00:00:00,2158-07-31,2158-07-31 20:52:00,Radiology,[**2158-7-31**] 8:52 PM\n CTA HEAD W&W/O C & R...


In [27]:
final_df.describe()

,ROW_ID,SUBJECT_ID,HADM_ID
count,17400.000000,17400.000000,17400.000000
mean,354063.750862,43463.657126,150011.170977
std,187887.469531,29942.677928,28880.720634
min,18.000000,4.000000,100003.000000
25%,195866.250000,17585.750000,124880.250000
50%,376292.500000,32594.000000,150361.500000
75%,516847.500000,69793.000000,174934.500000
max,651025.000000,99992.000000,199995.000000


In [28]:
notes_df.describe()

,SUBJECT_ID,HADM_ID
count,367101.000000,367101.000000
mean,40224.769469,149816.282247
std,29469.560491,28906.314246
min,4.000000,100003.000000
25%,15749.000000,125092.000000
50%,29664.000000,149397.000000
75%,65759.000000,175022.000000
max,99992.000000,199995.000000


In [29]:
print(len(final_df["SUBJECT_ID"].unique()))
print(len(notes_df["TEXT"].unique()))
print(len(final_df["HADM_ID"].unique()))
print(len(notes_df["HADM_ID"].unique()))

13952
367101
17400
17283


In [ ]:
notes_df_text = notes_df["TEXT"]
import time
substrings = []
x=1

def create_accum():
   counts = {}
   def accum(text):
        try:
            for x in text.split():
                if x in counts:
                    counts[x] += 1
                else: 
                    counts[x] = 1
        except:
            pass
   def fetch_accum():
        return counts
  
   return accum, fetch_accum

def second_accum(unique_dict):
    counts = {}
    def accum(text):
        try:
            start=time.time()
            alr =[]
            for sub in text.split():
                if sub in counts:
                    if sub not in alr:
                        counts[sub] += 1
                        alr.append(sub)
                else:
                    counts[sub] = 1
                    alr.append(sub)
            end = time.time()
            if ((end-start)>0.2):
                print(end-start)
        except:
            pass
    def fetch():
        return counts
    
    return accum, fetch
                    
    
# overall occurennce
print("overall occurrence start")
full_count, fetch = create_accum()
notes_df_text.apply(full_count)
full_count_sub = fetch()
print("overall occurrence end")


#notes occurrence
print("notes occurrence start")
notes_count, fetch = second_accum(full_count_sub)
notes_df_text.apply(notes_count)
notes_count_sub = fetch()
print("notes_occurrence end")



def third_accum():
    counts = {}
    
    biglistcounter = []
    def accum(subject):
        ind=0
        for subjectID in subject:
            start = time.time()
            tracker = []

            notes_text = notes_df[notes_df["SUBJECT_ID"]==subjectID]["TEXT"]
            for text in notes_text:
                try:
                    
                    for sub in text.split():
                        if (sub in counts):
                            if (sub not in tracker):
                                counts[sub]+=1
                                tracker.append(sub)
                        else:
                            counts[sub] = 1
                            tracker.append(sub)
                except:
                    pass
            end = time.time()
            ind+=1
            if(ind%1000==0):
                print((end-start)*(13952-ind)/3600)
    def fetch():
        return counts
    
    return accum, fetch

print("patients occurrence start")
dataf = notes_df["SUBJECT_ID"].unique()
print(dataf)
patient_count, fetch = third_accum()
patient_count(dataf)
patient_count_sub = fetch()
print(len(patient_count_sub))

In [36]:
#for some reason patient_count_sub did not register 80 words
notin = []
for sub in full_count_sub:
    if sub not in patient_count_sub:
        notin.append(sub)
dictappend = {}
for x in notin:
    try:
        dictappend[x] = len(notes_df[notes_df["TEXT"].str.contains(str(x), na=False, regex=False)]["SUBJECT_ID"].unique())
    except Exception as e:
        print(str(e) , x)
    

patient_count_sub_mod = {}
for key in full_count_sub:
    if key in patient_count_sub:
        patient_count_sub_mod[key] = patient_count_sub[key]
    else:
        patient_count_sub_mod[key] = dictappend[key]
len(patient_count_sub_mod)
with open('thirdaccumDOD.pickle', 'wb') as f:
    pickle.dump(patient_count_sub_mod, f)


print(len(full_count_sub),len(notes_count_sub),len(patient_count_sub_mod))

In [1]:
#while loading = 3
import pickle

with open('pickles/firstaccumDOD.pickle',"rb") as f:
    full_count_sub = pickle.load(f)
with open('pickles/secondaccumDOD.pickle',"rb") as f:
    notes_count_sub = pickle.load(f)
with open('pickles/thirdaccumDOD.pickle',"rb") as f:
    patient_count_sub_mod = pickle.load(f)

In [ ]:
#general population count - comparison
import time
# overall occurennce

def create_accum():
   counts = {}
   def accum(text):
        try:
            for x in text.split():
                if x in counts:
                    counts[x] += 1
                else: 
                    counts[x] = 1
        except:
            pass
   def fetch_accum():
        return counts
  
   return accum, fetch_accum

def second_accum(unique_dict):
    counts = {}
    def accum(text):
        try:
            start=time.time()
            alr =[]
            for sub in text.split():
                if sub in counts:
                    if sub not in alr:
                        counts[sub] += 1
                        alr.append(sub)
                else:
                    counts[sub] = 1
                    alr.append(sub)
            end = time.time()
            if ((end-start)>0.2):
                print(end-start)
        except:
            pass
    def fetch():
        return counts
    
    return accum, fetch


print("overall occurrence start")
general_count, fetch = create_accum()
noteevents["TEXT"].apply(general_count)
general_count_sub = fetch()
print("overall occurrence end")

with open('firstaccumDODgen.pickle', 'wb') as f:
    pickle.dump(general_count_sub, f)


#notes occurrence
print("notes occurrence start")
general_notes_count, fetch = second_accum(full_count_sub)
noteevents["TEXT"].apply(general_notes_count)
general_notes_count_sub = fetch()
print("notes_occurrence end")

with open('secondaccumDODgen.pickle', 'wb') as f:
    pickle.dump(general_notes_count_sub, f)

def third_accum_all():
    counts = {}
    
    biglistcounter = []
    def accum(subject):
        indec = 1
        for subjectID in subject:
            start = time.time()
            tracker = []

            notes_text = noteevents[noteevents["SUBJECT_ID"]==subjectID]["TEXT"]
            for text in notes_text:
                try:
                    
                    for sub in text.split():
                        if (sub in counts):
                            if (sub not in tracker):
                                counts[sub]+=1
                                tracker.append(sub)
                        else:
                            counts[sub] = 1
                            tracker.append(sub)
                except:
                    pass
            end = time.time()
            indec+=1
            if indec%100000 ==0:
                print((end-start)*(46146-indec)/3600)
    def fetch():
        return counts
    
    return accum, fetch

print("patients occurrence start")
data = noteevents["SUBJECT_ID"].unique()
print(data)
general_patient_count, fetch = third_accum_all()
print("2nd")
general_patient_count(data)
print("3rd")
general_patient_count_sub = fetch()
print("4th")

with open('thirdaccumDODgen.pickle', 'wb') as f:
    pickle.dump(general_patient_count_sub, f)

In [7]:
#while loading = 4
with open('pickles/firstaccumDODgen.pickle',"rb") as f:
    general_count_sub = pickle.load(f)
with open('pickles/secondaccumDODgen.pickle',"rb") as f:
    general_notes_count_sub = pickle.load(f)
with open('pickles/thirdaccumDODgen.pickle',"rb") as f:
    general_patient_count_sub = pickle.load(f)

In [48]:
#while loading = 5
print(len(general_count_sub),len(general_patient_count_sub),len(general_notes_count_sub))
general_combined_dict = {"strings":list(general_count_sub.keys()),"Full_Count":list(general_count_sub.values()), "Notes_Count":list(general_notes_count_sub.values()),"Patient_Count":list(general_patient_count_sub.values())}
general_all_counts = pd.DataFrame(general_combined_dict)
general_all_counts.nlargest(20,'Full_Count')

3793238 3793238 3793238


,strings,Full_Count,Notes_Count,Patient_Count
54,and,9842245,1534841,46097
59,to,8672697,1507053,46110
52,of,7905694,1512234,45956
21,the,7212747,1125181,44531
26,with,6791111,1511060,46007
20,in,4663945,1328539,45778
113,is,4591384,1123977,173
188,for,4568735,1309888,7
1322,-,3975669,339617,3381
119,on,3885019,1134734,31775


In [49]:
#while loading = 6
#DoD filtered - real
combined_dict = {"strings":list(full_count_sub.keys()),"Full_Count":list(full_count_sub.values()),"Notes_Count_Sub":list(notes_count_sub.values()),"Patient_Count_Sub":list(patient_count_sub_mod.values())}
all_counts = pd.DataFrame(combined_dict)
all_counts.head(20)

,strings,Full_Count,Notes_Count_Sub,Patient_Count_Sub
0,Admission,28856,25359,13597
1,Date:,35469,17796,13525
2,[**2141-12-8**],51,20,8
3,Discharge,94446,17898,13509
4,[**2141-12-14**],40,14,6
5,Date,24909,21904,12895
6,of,1779981,287888,13845
7,Birth:,16440,16429,12665
8,[**2117-8-7**],37,30,6
9,Sex:,16442,16429,12668


In [50]:
all_counts.nlargest(10,"Full_Count")[["strings","Full_Count"]]

,strings,Full_Count
61,and,2091574
71,to,1794816
6,of,1779981
72,the,1712929
49,with,1424280
43,is,1001571
66,for,988169
115,in,945057
89,on,801443
192,-,786137


In [51]:
all_counts.nsmallest(10,"Full_Count")[["strings","Full_Count"]]

,strings,Full_Count
380,165/120,1
470,Amylase-347*,1
486,Folate-18.5,1
601,Abdomin/Pelvis,1
929,Prefer:,1
1034,by:[**2141-12-16**],1
1121,[**2143-4-20**],1
1156,"182/102,",1
1175,800unit,1
1205,135-182/88-125.,1


In [52]:
all_counts.nlargest(10,"Notes_Count_Sub")[["strings","Notes_Count_Sub"]]

,strings,Notes_Count_Sub
6,of,287888
61,and,278113
49,with,277260
71,to,272161
66,for,245811
115,in,233387
72,the,231441
43,is,225319
89,on,202184
681,at,179336


In [53]:
all_counts.nsmallest(10,"Notes_Count_Sub")[["strings","Notes_Count_Sub"]]

,strings,Notes_Count_Sub
380,165/120,1
470,Amylase-347*,1
486,Folate-18.5,1
533,[**2141-12-10**]):,1
543,[**2141-12-8**]):,1
601,Abdomin/Pelvis,1
929,Prefer:,1
1034,by:[**2141-12-16**],1
1121,[**2143-4-20**],1
1156,"182/102,",1


In [54]:
all_counts.nlargest(10,"Patient_Count_Sub")[["strings","Patient_Count_Sub"]]

,strings,Patient_Count_Sub
61,and,13847
6,of,13845
71,to,13845
49,with,13843
72,the,13841
66,for,13838
115,in,13838
43,is,13828
89,on,13821
168,no,13814


In [55]:
all_counts.nsmallest(10,"Patient_Count_Sub")[["strings","Patient_Count_Sub"]]

,strings,Patient_Count_Sub
133,"240/184,",1
149,"intropaste,",1
217,CKD/ESRD:,1
218,Diagosed,1
258,plasmapheresisis),1
267,([**2139**]);,1
271,"([**4-/2138**],",1
273,[**9-/2140**]),1
279,([**2137**]-[**2140**]),1
283,[**8-/2140**]),1


In [56]:
#DoD
import math
def getTFIDF(total,column):
    tfidf_list = []
    for x in all_counts.index:
        start = time.time()
        row = all_counts.iloc[x]
        full_count = row["Full_Count"]
        count_number = row[column]
        tfidf = full_count * math.log(total/count_number)
        tfidf_list.append(tfidf)
        end = time.time()
        if ((x+1)%100000==0):
            print(x, end-start)
    return tfidf_list
print("notes")
notes_tfidf = getTFIDF(len(notes_df_text.index),"Notes_Count_Sub")
print("subjects")
subject_tfidf = getTFIDF(len(notes_df["SUBJECT_ID"].unique()), "Patient_Count_Sub")

notes
99999 0.0001857280731201172
199999 0.0001742839813232422
299999 0.0001800060272216797
399999 0.0002548694610595703
499999 0.00017786026000976562
599999 0.0001785755157470703
699999 0.00019097328186035156
799999 0.00017642974853515625
899999 0.00018596649169921875
999999 0.00018358230590820312
1099999 0.0002079010009765625
1199999 0.0002110004425048828
1299999 0.0002224445343017578
1399999 0.00019693374633789062
1499999 0.00018143653869628906
subjects
99999 0.00017571449279785156
199999 0.0001811981201171875
299999 0.00018334388732910156
399999 0.00017213821411132812
499999 0.0001881122589111328
599999 0.00018024444580078125
699999 0.00018215179443359375
799999 0.0002086162567138672
899999 0.00017976760864257812
999999 0.00019598007202148438
1099999 0.00021195411682128906
1199999 0.00020194053649902344
1299999 0.0003006458282470703
1399999 0.00019860267639160156
1499999 0.00021123886108398438


In [ ]:
#general
print(len(general_all_counts.index))

def getTFIDF_general(total,column):
    tfidf_list = []
    for x in general_all_counts.index:
        start = time.time()
        row = general_all_counts.iloc[x]
        full_count = row["Full_Count"]
        count_number = row[column]
        tfidf = full_count * math.log(total/count_number)
        tfidf_list.append(tfidf)
        end = time.time()
        if ((x+1)%100000==0):
            print(x, end-start)
    return tfidf_list
general_notes_tfidf = getTFIDF_general(len(general_all_counts.index),"Notes_Count")
print("next")


print("in")
general_patient_tfidf = getTFIDF_general(len(noteevents["SUBJECT_ID"].unique()),"Patient_Count")

In [59]:
#Dod


tfidf_df = pd.DataFrame({"strings":all_counts["strings"].copy(),"notesTFIDF":notes_tfidf,"patientTFIDF": subject_tfidf})

stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such','only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
ind = 0
removed_words = tfidf_df[tfidf_df["strings"].str.lower().isin(stop_words)==True]

tfidf_df = tfidf_df[tfidf_df["strings"].str.lower().isin(stop_words)==False]
tfidf_df_bool = tfidf_df["strings"].str.contains(r'[A-Za-z0-9]+')
removed_words = pd.concat([tfidf_df[tfidf_df_bool == False], removed_words])
tfidf_df = tfidf_df[tfidf_df_bool]
print(removed_words)
print(len(tfidf_df.index))

        strings    notesTFIDF   patientTFIDF
15            /  6.727540e+05  181581.766465
25            &  1.787946e+05   24439.994127
42          **]  3.590725e+05    3311.049464
192           -  1.342721e+06   70992.131812
328           .  5.499147e+05   79430.886823
...         ...           ...            ...
1418119     noW  1.281339e+01       9.536113
1442955      iF  1.281339e+01       9.536113
1451054  Myself  1.281339e+01       9.536113
1473910      dO  1.281339e+01       9.536113
1488605      wE  1.281339e+01       9.536113

[1799 rows x 3 columns]
1547590


In [60]:
len(all_counts["strings"])-len(tfidf_df["strings"])


1799

In [10]:
#while loading = 7
tfidf_df = pd.read_pickle('pickles/tfidf_df.pickle')

In [65]:
total_notes_tfidf = tfidf_df["notesTFIDF"].sum()
total_patient_tfidf = tfidf_df["patientTFIDF"].sum()
print(0.8*total_notes_tfidf,0.8*total_patient_tfidf)

def getactivation(total,column,start,stop):
    activation_notes = int()
    for x in range(start,stop):
        sums = tfidf_df.nlargest(x,column)[column].sum()
        if sums >= (0.8*total):
            activation_notes = x
            break
    return activation_notes

notes_act = getactivation(total_notes_tfidf,"notesTFIDF",39800,40000)
notes_act

patient_act = getactivation(total_patient_tfidf,"patientTFIDF",89500,90000)
patient_act

tfidf_df.nlargest(88000,"patientTFIDF")["patientTFIDF"].sum()


In [66]:
#general

tfidf_df_general = pd.DataFrame({"strings":general_all_counts["strings"].copy(),"notesTFIDF":general_notes_tfidf,"patientTFIDF": general_patient_tfidf})

removed_words_general = tfidf_df_general[tfidf_df_general["strings"].str.lower().isin(stop_words)==True]

tfidf_df_general = tfidf_df_general[tfidf_df_general["strings"].str.lower().isin(stop_words)==False]
tfidf_df_general_bool = tfidf_df_general["strings"].str.contains(r'[A-Za-z0-9]+')
removed_words_general = pd.concat([tfidf_df_general[tfidf_df_general_bool == False], removed_words_general])
tfidf_df_general = tfidf_df_general[tfidf_df_general_bool]
print(len(tfidf_df_general.index))
print(len(general_all_counts.index)-len(tfidf_df_general.index))

3790360
2878


In [67]:
tfidf_df_general

,strings,notesTFIDF,patientTFIDF
0,Admission,3.960154e+05,7734.538159
1,Date:,5.004676e+05,13941.094373
2,[**2151-7-16**],2.007316e+03,1422.625298
3,Discharge,1.087182e+06,30807.322128
4,[**2151-8-4**],5.487733e+02,366.480935
...,...,...,...
3793233,[**2114**].(,1.514873e+01,10.739566
3793234,plannned.,1.514873e+01,10.739566
3793235,+milia,1.514873e+01,10.739566
3793236,37-->Start,1.514873e+01,10.739566


In [68]:
tfidf_df_general.to_pickle("tfidf_df_general.pickle")

In [11]:
#while loading = 8
tfidf_df_general = pd.read_pickle('pickles/tfidf_df_general.pickle')

In [72]:
total_notes_tfidf_general = tfidf_df_general["notesTFIDF"].sum()
total_patient_tfidf_general = tfidf_df_general["patientTFIDF"].sum()
print(0.8*total_notes_tfidf_general,0.8*total_patient_tfidf_general)

def getactivation_general(total,column,start,stop):
    activation_notes = int()
    for x in range(start,stop):
        sums = tfidf_df_general.nlargest(x,column)[column].sum()
        if sums >= (0.8*total):
            activation_notes = x
            break
    return activation_notes
print("notes")
notes_act_general = getactivation_general(total_notes_tfidf_general,"notesTFIDF",35700,35800)
print(notes_act_general)
patient_act_general = getactivation_general(total_patient_tfidf_general,"patientTFIDF",26000,27000)
print(patient_act_general)

notes_act_general

tfidf_df_general.nlargest(35700,"notesTFIDF")["notesTFIDF"].sum()


In [73]:
#DoD

words_list_notes = tfidf_df.nlargest(notes_act,"notesTFIDF")
words_list_notes = words_list_notes[["strings","notesTFIDF"]]
notes_percentages = []
totalfornotes = all_counts["Notes_Count_Sub"].sum()
for coun in all_counts["Notes_Count_Sub"]:
    notes_percentages.append(coun/totalfornotes*100)

    notes_pdf = pd.DataFrame({"strings":all_counts["strings"].copy(), "Percent":notes_percentages})
words_list_notes = words_list_notes.merge(notes_pdf)


words_list_notes_50 = words_list_notes.iloc[0:50]
words_list_notes_50

ehere
here
p much done


,strings,notesTFIDF,Percent
0,mL,721288.849261,0.059009
1,mg,622418.030272,0.045819
2,Tablet,594890.967871,0.018041
3,Sig:,497055.795243,0.016334
4,PO,481798.360016,0.046009
5,(1),436165.261866,0.033198
6,BLOOD,388561.166828,0.020202
7,Pt,373435.153615,0.117029
8,left,369760.252491,0.157988
9,PM,367138.596667,0.133130


In [74]:
words_list_patient = tfidf_df.nlargest(patient_act,"patientTFIDF")
words_list_patient = words_list_patient[["strings","patientTFIDF"]]
patient_percentages = []
totalforpatient = all_counts["Patient_Count_Sub"].sum()
for coun in all_counts["Patient_Count_Sub"]:
    patient_percentages.append(coun/totalforpatient*100)
print("here")
patient_pdf = pd.DataFrame({"strings":all_counts["strings"].copy(), "Percent":patient_percentages})
words_list_patient = words_list_patient.merge(patient_pdf)
words_list_patient_50 = words_list_patient.iloc[0:50]
words_list_patient_50

here


,strings,patientTFIDF,Percent
0,mL,233243.868264,0.021237
1,mEq/L,165998.346756,0.010178
2,mg/dL,165807.097518,0.010557
3,No(t),158620.876734,0.006868
4,Comments:,124784.702417,0.010380
5,Response:,118970.452587,0.010365
6,Action:,118311.762558,0.010358
7,Assessment:,115794.181496,0.011065
8,Sounds:,104654.893299,0.009529
9,cmH2O,104412.954385,0.005390


In [75]:
print(words_list_patient_50["strings"].tolist())

['mL', 'mEq/L', 'mg/dL', 'No(t)', 'Comments:', 'Response:', 'Action:', 'Assessment:', 'Sounds:', 'cmH2O', 'Total', 'pulse:', 'Not', 'Lung', 'tube', 'Plan:', 'mg/dL,', 'Assessment', 'Pt', 'medications:', 'K/uL', 'assessment:', 'trach', 'SpO2:', 'Airway', 'valve', 'PICC', 'pt', 'Cuff', 'Respiratory', 'Order', 'Ideal', 'woman', 'Tablet', 'BLOOD', 'man', 'liver', 'breathing', 'vent', 'Sig:', 'Continue', 'C', 'Data', 'Sputum', 'aortic', 'w/', 'ICU', 'renal', 'kg', 'Day']


In [76]:
print(words_list_notes_50["strings"].tolist())

['mL', 'mg', 'Tablet', 'Sig:', 'PO', '(1)', 'BLOOD', 'Pt', 'left', 'PM', 'right', 'Name', 'patient', 'One', 'no', 'No(t)', 'pt', 'No', 'CT', 'mg/dL', 'mEq/L', 'Discharge', 'not', 'valve', '[**Last', 'tube', 'aortic', 'IV', 'pain', 'Patient', 'Not', 'ICU', 'ventricular', 's/p', '[**Hospital1', 'given', 'Lung', 'normal', 'w/', 'chest', 'Comments:', '2', '1', 'Normal', 'Plan:', 'Assessment:', 'Action:', 'Response:', 'evidence', 'acute']


In [77]:
#general
words_list_notes_general = tfidf_df_general.nlargest(notes_act_general,"notesTFIDF")
words_list_notes_general = words_list_notes_general[["strings","notesTFIDF"]]
print(words_list_notes_general)
notes_percentages_general = []
print("ehere")
totalfornotes_general = general_all_counts["Notes_Count"].sum()
for coun in general_all_counts["Notes_Count"]:
    notes_percentages_general.append(coun/totalfornotes_general*100)
notes_pdf_general = pd.DataFrame({"strings":general_all_counts["strings"].copy(), "Percent":notes_percentages_general})
words_list_notes_general = words_list_notes_general.merge(notes_pdf_general)

print("p much done")
words_list_notes_50_general = words_list_notes_general.iloc[0:50]
words_list_notes_50_general

            strings    notesTFIDF
939              mL  5.095955e+06
38               no  2.992758e+06
486              No  2.892433e+06
2468             PM  2.835077e+06
22             left  2.805587e+06
...             ...           ...
41395    [**3-4**].  5.479935e+03
54808       HEARING  5.479935e+03
14878          (WBC  5.479074e+03
1207581        tnsf  5.478937e+03
230408     Q6H:PRN,  5.478799e+03

[35723 rows x 2 columns]
ehere
p much done


,strings,notesTFIDF,Percent
0,mL,5.095955e+06,0.053321
1,no,2.992758e+06,0.283595
2,No,2.892433e+06,0.261909
3,PM,2.835077e+06,0.130249
4,left,2.805587e+06,0.156982
5,mg,2.702998e+06,0.039541
6,Pt,2.648502e+06,0.110702
7,Name,2.645391e+06,0.184958
8,right,2.642388e+06,0.145411
9,patient,2.200833e+06,0.089346


In [78]:
words_list_patient_general = tfidf_df_general.nlargest(patient_act_general,"patientTFIDF")
words_list_patient_general = words_list_patient_general[["strings","patientTFIDF"]]
patient_percentages_general = []
print("ehere")
totalforpatient_general = general_all_counts["Patient_Count"].sum()
for coun in general_all_counts["Patient_Count"]:
    patient_percentages_general.append(coun/totalforpatient_general*100)
print("here")
patient_pdf_general = pd.DataFrame({"strings":general_all_counts["strings"].copy(), "Percent":patient_percentages_general})
words_list_patient_general = words_list_patient_general.merge(patient_pdf_general)

print("p much done")
words_list_patient_50_general = words_list_patient_general.iloc[0:50]
words_list_patient_50_general

ehere
here
p much done


,strings,patientTFIDF,Percent
0,mL,8.277837e+06,3.001661e-04
1,Clip,5.983388e+06,8.751200e-07
2,Plan:,5.291093e+06,7.000960e-06
3,Action:,5.057327e+06,1.750240e-06
4,REASON,4.711700e+06,5.250720e-06
5,PM,4.612033e+06,1.274175e-03
6,small,4.226380e+06,5.250720e-06
7,mEq/L,4.133896e+06,3.237944e-05
8,2**],4.062810e+06,8.751200e-07
9,Not,4.004075e+06,8.751200e-07


In [79]:
print(words_list_patient_50_general["strings"].tolist())

['mL', 'Clip', 'Plan:', 'Action:', 'REASON', 'PM', 'small', 'mEq/L', '2**]', 'Not', 'IMPRESSION:', 'Response:', '[**Hospital1', 'remains', 'Assessment:', '(Radiology)', 'Total', 'hours', 'post', 'Plan', 'Comments:', 'mg/dL,', '[**Clip', 'Infant', 'evidence', 'noted.', 'REPORT', 'increased', 'Last', '(STitle)', 'Lung', 'Hct', 'seen', 'right', 'NO', 'Continue', 'Cont', 'acute', 'IV', 'COMPARISON:', 'continue', 'Respiratory', 'Heart', 'received', 'pt', 'not', 'mg/dL', 'BS', 'level', 'atrial']


In [80]:
print(words_list_notes_50_general["strings"].tolist())

['mL', 'no', 'No', 'PM', 'left', 'mg', 'Pt', 'Name', 'right', 'patient', 'pt', 'CT', 'Tablet', 'PO', 'not', 'mg/dL', 'mEq/L', '[**Last', 'No(t)', 'Sig:', '(1)', 's/p', 'IV', 'valve', 'Plan:', 'tube', 'Assessment:', 'given', 'Response:', 'Action:', 'ICU', 'aortic', 'ventricular', 'Not', 'w/', 'Continue', 'normal', 'Patient', 'BLOOD', 'pain', '(NI)', 'chest', '[**Hospital', '24', '2', 'One', 'prior', 'noted', 'Reason:', 'Number']


In [81]:
l = [words_list_patient_general,
words_list_notes_general,
words_list_patient,
words_list_notes]
for lists in l:
    print(len(lists))

26622
35723
89610
39946


In [ ]:
tot_patient_general = words_list_patient_general["patientTFIDF"].sum()
tot_patient = words_list_patient["patientTFIDF"].sum()
patient_salience = {}
indasdf=0
for word in words_list_patient["strings"]:
    indasdf+=1
    dod = float(words_list_patient[words_list_patient["strings"]==word]["patientTFIDF"])
    gen = words_list_patient_general[words_list_patient_general["strings"]==word]
    if not gen.empty:
        gen = float(gen["patientTFIDF"])
        patient_salience[word] = dod/tot_patient-gen/tot_patient_general
    else:
        gen = 0
        patient_salience[word] = dod/tot_patient-gen/tot_patient_general



In [12]:
#while loading = 9
salience_notes = pd.read_pickle('pickles/salience_notes.pickle')
salience_patient = pd.read_pickle('pickles/salience_patient.pickle')
salience_notes.nlargest(20,'salience_values')

,strings,salience_values
2,Tablet,0.000701
3,Sig:,0.000592
6,BLOOD,0.000477
1,mg,0.000424
5,(1),0.000415
13,One,0.000407
4,PO,0.000402
21,Discharge,0.000287
36,Lung,0.000240
60,DAILY,0.000210


In [83]:
salience_patient = pd.DataFrame({"strings":list(patient_salience.keys()),"salience_values":list(patient_salience.values())})
salience_patient

,strings,salience_values
0,mL,-0.004019
1,mEq/L,-0.001659
2,mg/dL,-0.000054
3,No(t),0.000033
4,Comments:,-0.001148
...,...,...
87856,-Lacunar,0.000002
87857,"Montaggeia,",0.000002
87858,"Magma,CABGx2(SVG-OM,SVG-PDA),",0.000002
87859,[**2190-10-30**].,0.000002


In [84]:
salience_patient.nlargest(50,'salience_values')

,strings,salience_values
8,Sounds:,0.000434
22,trach,0.000406
26,PICC,0.000388
24,Airway,0.000352
37,breathing,0.000302
43,Sputum,0.000285
49,Day,0.000250
36,liver,0.000250
51,ventilation,0.000245
107,intubated,0.000239


In [85]:
tot_notes_general = words_list_notes_general["notesTFIDF"].sum()
tot_notes = words_list_notes["notesTFIDF"].sum()
notes_salience = {}
for word in words_list_notes["strings"]:
    dod = float(words_list_notes[words_list_notes["strings"]==word]["notesTFIDF"])
    gen = words_list_notes_general[words_list_notes_general["strings"]==word]
    if not gen.empty:
        gen = float(gen["notesTFIDF"])
        notes_salience[word] = dod/tot_notes-gen/tot_notes_general
    else:
        gen = 0
        notes_salience[word] = dod/tot_notes-gen/tot_notes_general

salience_notes = pd.DataFrame({"strings":list(notes_salience.keys()),"salience_values":list(notes_salience.values())})
salience_notes

,strings,salience_values
0,mL,-0.000706
1,mg,0.000424
2,Tablet,0.000701
3,Sig:,0.000592
4,PO,0.000402
...,...,...
39941,"finger-nose-finger,",0.000003
39942,87%.,0.000003
39943,CALCIUM-9.1,0.000003
39944,~4,-0.000002


In [86]:
salience_notes.nlargest(50,'salience_values')

,strings,salience_values
2,Tablet,0.000701
3,Sig:,0.000592
6,BLOOD,0.000477
1,mg,0.000424
5,(1),0.000415
13,One,0.000407
4,PO,0.000402
21,Discharge,0.000287
36,Lung,0.000240
60,DAILY,0.000210
